### Gaussian Naive Bayes Classifier
GaussianNB implements the Gaussian Naive Bayes algorithm for classification. The likelihood of the features is assumed to be Gaussian:

$$
P(x_i \mid y) = \frac{1}{\sqrt{2\pi\sigma^2_y}} \exp\left(-\frac{(x_i - \mu_y)^2}{2\sigma^2_y}\right)
$$

The parameters $\sigma_y$ and $\mu_y$ are estimated using maximum likelihood.


```python
class sklearn.naive_bayes.GaussianNB(*, priors=None, var_smoothing=1e-09)
```

#### Variables
- **prior:** Probabilities of the classes. If specified, the priors are not adjusted according to the data. Otherwise, calculate prior from existing data.
- **var_smoothing:** Portion of the largest variance of all features that's added to variances for caclulation stability.

#### 1 Generate Gaussian blobs

In [10]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

# Generate random 100 2-d datapoints
X, y = make_blobs(100, 2, centers=2, random_state=2, cluster_std=1.5)

# Display data with plots
fig, ax = plt.subplots()

ax.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='RdBu')

xlim = (-8, 8)
ylim = (-15, 5)

xg = np.linspace(xlim[0], xlim[1], 60)
yg = np.linspace(ylim[0], ylim[1], 40)
xx, yy = np.meshgrid(xg, yg)
Xgrid = np.vstack([xx.ravel(), yy.ravel()]).T

for label, color in enumerate(['red', 'blue']):
    mask = (y == label)
    mu, std = X[mask].mean(0), X[mask].std(0)
    P = np.exp(-0.5 * (Xgrid - mu) ** 2 / std ** 2).prod(1)
    Pm = np.ma.masked_array(P, P < 0.03)
    ax.pcolorfast(xg, yg, Pm.reshape(xx.shape), alpha=0.5,
                  cmap=color.title() + 's')
    ax.contour(xx, yy, P.reshape(xx.shape),
               levels=[0.01, 0.1, 0.5, 0.9],
               colors=color, alpha=0.2)

ax.set(xlim=xlim, ylim=ylim)


[(-8.0, 8.0), (-15.0, 5.0)]

The ellipses here represent the Gaussian generative model for each label, with larger probability toward the center of the ellipses. With this generative model in place for each class, we have a simple recipe to compute the likelihood for any data point, and thus we can quickly compute the posterior ratio and determine which label is the most probable for a given point.

#### 2 GaussianNB
This procedure is implemented in Scikit-Learn's sklearn.naive_bayes.GaussianNB estimator:

In [21]:
%matplotlib inline
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
clf.fit(X, y)

GaussianNB()

Now let's generate some new data and predict the label:

In [22]:
rng = np.random.RandomState(0)
Xnew = [-6, -14] + [14, 18] * rng.rand(2000, 2)
ynew = clf.predict(Xnew)
print(ynew)

[1 1 1 ... 0 1 1]


Then we can plot this new data to get an idea of where the decision boundary is:

In [25]:
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='RdBu')
lim = plt.axis()
plt.scatter(Xnew[:, 0], Xnew[:, 1], c=ynew, s=20, cmap='RdBu', alpha=0.1)
plt.axis(lim);

We see a slightly curved boundary in the classifications—in general, the boundary in Gaussian naive Bayes is quadratic.

### Example: Classify iris data

#### 1. Load real world dataset
The [sklearn.datasets](https://scikit-learn.org/stable/datasets.html#datasets) package embeds some small toy datasets. This package also features helpers to fetch larger datasets commonly used by the machine learning community to benchmark algorithms on data that comes from the ‘real world’.

In [5]:
from sklearn.datasets import load_iris

X, y = load_iris(return_X_y=True)
print("X1:", X[0])
print("y1:", y[0])

X1: [5.1 3.5 1.4 0.2]
y1: 0


#### 2. Split dataset into random train and test subsets

```python
sklearn.model_selection.train_test_split(*arrays, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None)
```

Parameters:
- *arrays: sequence of indexables with same length / shape[0]  
Allowed inputs are lists, numpy arrays, scipy-sparse matrices or pandas dataframes.

- test_size: float or int, default=None  
If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the test split. If int, represents the absolute number of test samples. If None, the value is set to the complement of the train size. If train_size is also None, it will be set to 0.25.

- train_size: float or int, default=None  
If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the train split. If int, represents the absolute number of train samples. If None, the value is automatically set to the complement of the test size.

- random_state: int, RandomState instance or None, default=None
Controls the shuffling applied to the data before applying the split. Pass an int for reproducible output across multiple function calls.

- shuffle: bool, default=True
Whether or not to shuffle the data before splitting. If shuffle=False then stratify must be None.

Returns:  
X_train, X_test, y_train, y_test

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
print("The number of train subset:", X_train.shape[0])
print("The number of test subset:", X_test.shape[0])

The number of train subset: 75
The number of test subset: 75


#### 3. GaussianNB
Please use a Gaussian Naive Bayes Classifier to classify the iris dataset and evaluate the accuracy of prediction in the test set.

```python
sklearn.metrics.accuracy_score(y_true, y_pred, *, normalize=True, sample_weight=None)
```

Parameters:
- y_true: 1d array-like, or label indicator array / sparse matrix  
Ground truth (correct) labels.

- y_pred: 1d array-like, or label indicator array / sparse matrix
Predicted labels, as returned by a classifier.

- normalize: bool, default=True  
If False, return the number of correctly classified samples. Otherwise, return the fraction of correctly classified samples.

Returns:
- score: float  
If normalize == True, return the fraction of correctly classified samples (float), else returns the number of correctly classified samples (int).  
The best performance is 1 with normalize == True and the number of samples with normalize == False.

In [7]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

clf = GaussianNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print('Accuracy is: ', accuracy_score(y_test,y_pred))

Accuracy is:  0.9466666666666667
